# Environment Settings

In [1]:
# pip install torch numpy pandas matplotlib seaborn scikit-learn

In [2]:
import torch
print(torch.version.cuda)


12.4


In [3]:
import torch

if torch.cuda.is_available():
    gpu_id = torch.cuda.current_device()
    gpu_name = torch.cuda.get_device_name(gpu_id)
    gpu_capability = torch.cuda.get_device_capability(gpu_id)
    total_memory = torch.cuda.get_device_properties(gpu_id).total_memory
    
    print(f"Total number of GPU: {torch.cuda.device_count()}")  # Number of GPUs available
    print(f"Total GPU memory: {total_memory / 1e9} GB")
    print(f"GPU ID: {gpu_id}")
    print(f"GPU Name: {gpu_name}")
    print(f"GPU Compute Capability: {gpu_capability}")
else:
    print("No GPU is available.")

Total number of GPU: 4
Total GPU memory: 51.033931776 GB
GPU ID: 0
GPU Name: NVIDIA RTX A6000
GPU Compute Capability: (8, 6)


In [ ]:
import torch

torch.cuda.set_device(3)

print("Current Device:", torch.cuda.current_device())
print("Device Name:", torch.cuda.get_device_name(torch.cuda.current_device()))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device


Current Device: 3
Device Name: NVIDIA RTX A6000


device(type='cuda')

In [5]:
import warnings
warnings.filterwarnings('ignore')

# Load Data

In [6]:
import pandas as pd
import numpy as np

In [7]:
data = pd.read_csv('../dataset/exchange_rate.csv', index_col='date', parse_dates=True)

In [8]:
data.head()

,0,1,2,3,4,5,6,OT
date,,,,,,,,
1990-01-01,0.7855,1.6110,0.861698,0.634196,0.211242,0.006838,0.525486,0.5930
1990-01-02,0.7818,1.6100,0.861104,0.633513,0.211242,0.006863,0.523972,0.5940
1990-01-03,0.7867,1.6293,0.861030,0.648508,0.211242,0.006975,0.526316,0.5973
1990-01-04,0.7860,1.6370,0.862069,0.650618,0.211242,0.006953,0.523834,0.5970
1990-01-05,0.7849,1.6530,0.861995,0.656254,0.211242,0.006940,0.527426,0.5985


In [9]:
print(data.dtypes)
print('shape: ', data.shape)

0     float64
1     float64
2     float64
3     float64
4     float64
5     float64
6     float64
OT    float64
dtype: object
shape:  (7588, 8)


In [10]:
target = data['OT']
features = data.drop(['OT'],axis=1)

In [11]:
features['DATE'] = features.index.strftime('%Y%m%d%H').astype(int)

In [12]:
target.index

DatetimeIndex(['1990-01-01', '1990-01-02', '1990-01-03', '1990-01-04',
               '1990-01-05', '1990-01-06', '1990-01-07', '1990-01-08',
               '1990-01-09', '1990-01-10',
               ...
               '2010-10-01', '2010-10-02', '2010-10-03', '2010-10-04',
               '2010-10-05', '2010-10-06', '2010-10-07', '2010-10-08',
               '2010-10-09', '2010-10-10'],
              dtype='datetime64[ns]', name='date', length=7588, freq=None)

In [13]:
import torch
import numpy as np
import random

def set_seed(seed=42):
    random.seed(seed)                           # Python random
    np.random.seed(seed)                        # NumPy
    torch.manual_seed(seed)                     # PyTorch CPU
    torch.cuda.manual_seed(seed)                # PyTorch GPU (single)
    torch.cuda.manual_seed_all(seed)            # PyTorch GPU (multi)
    torch.backends.cudnn.deterministic = True   # 연산 순서 고정
    torch.backends.cudnn.benchmark = False      # 성능 대신 일관성 선택

In [14]:
set_seed(24)

# Model Modification for DLinear (Decomposition-wise Look-ahead Augmentation)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

class moving_avg(nn.Module):

    def __init__(self, kernel_size, stride):
        super(moving_avg, self).__init__()
        self.kernel_size = kernel_size
        self.avg = nn.AvgPool1d(kernel_size=kernel_size, stride=stride, padding=0)

    def forward(self, x):
        # padding on the both ends of time series
        front = x[:, 0:1, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        end = x[:, -1:, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        x = torch.cat([front, x, end], dim=1)
        x = self.avg(x.permute(0, 2, 1))
        x = x.permute(0, 2, 1)
        return x


class series_decomp(nn.Module):

    def __init__(self, kernel_size):
        super(series_decomp, self).__init__()
        self.moving_avg = moving_avg(kernel_size, stride=1)

    def forward(self, x):
        moving_mean = self.moving_avg(x)
        res = x - moving_mean
        return res, moving_mean

class Model(nn.Module):

    def __init__(self, configs):
        super(Model, self).__init__()
        self.seq_len = configs.seq_len
        self.pred_len = configs.pred_len
        self.individual = configs.individual
        self.channels = configs.enc_in
        self.attach_to_trend = configs.attach_to_trend
        self.attach_to_seasonal = configs.attach_to_seasonal
        

        # Decompsition Kernel Size
        kernel_size = 25
        self.decompsition = series_decomp(kernel_size)
        

        self.seq_len_trend = self.seq_len + (self.pred_len // 3) if self.attach_to_trend else self.seq_len
        self.seq_len_seasonal = self.seq_len + (self.pred_len // 3) if self.attach_to_seasonal else self.seq_len

        
        if self.individual:
            self.Linear_Seasonal = nn.ModuleList()
            self.Linear_Trend = nn.ModuleList()
            
            for i in range(self.channels):
                self.Linear_Seasonal.append(nn.Linear(self.seq_len,self.pred_len))
                self.Linear_Trend.append(nn.Linear(self.seq_len,self.pred_len))

        else:
            self.Linear_Seasonal = nn.Linear(self.seq_len,self.pred_len)
            self.Linear_Trend = nn.Linear(self.seq_len,self.pred_len)
            

    def forward(self, x, ground_truth=None):
        seasonal_init, trend_init = self.decompsition(x)
        seasonal_init, trend_init = seasonal_init.permute(0,2,1), trend_init.permute(0,2,1)
        if ground_truth is not None:
            kernel_size = 25
            self.decompsition = series_decomp(kernel_size)
            ground_truth_seasonal, ground_truth_trend = self.decompsition(ground_truth)

            if self.seq_len_seasonal > self.seq_len:
                ground_truth_seasonal=ground_truth_seasonal.permute(0, 2, 1)
                seasonal_init = torch.cat([seasonal_init, ground_truth_seasonal], dim=2)
            if self.seq_len_trend > self.seq_len:
                ground_truth_trend=ground_truth_trend.permute(0, 2, 1)
                trend_init = torch.cat([trend_init, ground_truth_trend], dim=2)
        
        
        if self.individual:
            seasonal_output = torch.zeros([seasonal_init.size(0),seasonal_init.size(1),self.pred_len],dtype=seasonal_init.dtype).to(seasonal_init.device)
            trend_output = torch.zeros([trend_init.size(0),trend_init.size(1),self.pred_len],dtype=trend_init.dtype).to(trend_init.device)
            for i in range(self.channels):
                seasonal_output[:,i,:] = self.Linear_Seasonal[i](seasonal_init[:,i,:])
                trend_output[:,i,:] = self.Linear_Trend[i](trend_init[:,i,:])
        else:
            seasonal_output = self.Linear_Seasonal(seasonal_init)
            trend_output = self.Linear_Trend(trend_init)

        x = seasonal_output + trend_output
        return x.permute(0,2,1) 

# Split the Data

In [ ]:
import pandas as pd
import torch
import numpy as np
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset


target = data['OT'].values.reshape(-1, 1)  

seq_len  = 336
pred_len = 96

total_len  = len(target)
num_train  = int(total_len * 0.7)
num_test   = int(total_len * 0.2)
num_val    = total_len - num_train - num_test

border1s = [
    0,
    num_train - seq_len,
    total_len - num_test - seq_len
]
border2s = [
    num_train,
    num_train + num_val,
    total_len
]

train = target[border1s[0]:border2s[0]]
val   = target[border1s[1]:border2s[1]]
test  = target[border1s[2]:border2s[2]]

scaler = StandardScaler()
train = scaler.fit_transform(train)
val   = scaler.transform(val)
test  = scaler.transform(test)

def create_inout_sequences_univariate(data, seq_len, pred_len):
    seqs = []
    for i in range(len(data) - seq_len - pred_len + 1):
        seq_x = data[i : i + seq_len]
        seq_y = data[i + seq_len : i + seq_len + pred_len]
        seqs.append((seq_x, seq_y))
    return seqs

train_data = create_inout_sequences_univariate(train, seq_len, pred_len)
val_data   = create_inout_sequences_univariate(val,   seq_len, pred_len)
test_data  = create_inout_sequences_univariate(test,  seq_len, pred_len)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_sequences = torch.tensor([x[0] for x in train_data]).float().to(device)
train_labels    = torch.tensor([x[1] for x in train_data]).float().to(device)

val_sequences   = torch.tensor([x[0] for x in val_data]).float().to(device)
val_labels      = torch.tensor([x[1] for x in val_data]).float().to(device)

test_sequences  = torch.tensor([x[0] for x in test_data]).float().to(device)
test_labels     = torch.tensor([x[1] for x in test_data]).float().to(device)

print("Train Sequences:", train_sequences.shape)
print("Train Labels:   ", train_labels.shape)
print("Val Sequences:  ", val_sequences.shape)
print("Val Labels:     ", val_labels.shape)
print("Test Sequences: ", test_sequences.shape)
print("Test Labels:    ", test_labels.shape)


Train Sequences: torch.Size([4880, 336, 1])
Train Labels:    torch.Size([4880, 96, 1])
Val Sequences:   torch.Size([665, 336, 1])
Val Labels:      torch.Size([665, 96, 1])
Test Sequences:  torch.Size([1422, 336, 1])
Test Labels:     torch.Size([1422, 96, 1])


In [ ]:
batch_size = 8

# Train DataLoader
train_dataset = TensorDataset(train_sequences, train_labels)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Validation DataLoader
val_dataset = TensorDataset(val_sequences, val_labels)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Test DataLoader
test_dataset = TensorDataset(test_sequences, test_labels)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


# 1st Model Training (Early Stopping)

In [18]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error


In [19]:
import torch.optim as optim
import os

# Assuming that data is already scaled before being passed to the DataLoader
seq_len = 336
pred_len = 96  # Set pred_len to the desired prediction horizon

class Configs:
    def __init__(self, seq_len, pred_len, enc_in, individual, learning_rate, lradj, patience, save_path, attach_to_trend, attach_to_seasonal):
        self.seq_len = seq_len
        self.pred_len = pred_len  # Set pred_len appropriately
        self.enc_in = enc_in
        self.individual = individual
        self.learning_rate = learning_rate
        self.lradj = lradj
        self.patience = patience
        self.save_path = save_path
        self.attach_to_trend = attach_to_trend
        self.attach_to_seasonal = attach_to_seasonal

configs = Configs(seq_len=seq_len, pred_len=pred_len, enc_in=1, individual=False,
                   learning_rate=0.0005, lradj='type1',  patience=3, save_path="./model_ER", attach_to_trend=False , attach_to_seasonal=False) # 원하는 값으로 설정


model = Model(configs).to(device)
loss_function = nn.MSELoss()  # Mean Squared Error loss
optimizer = optim.Adam(model.parameters(), lr=configs.learning_rate)

if not os.path.exists(configs.save_path):
    os.makedirs(configs.save_path)

# EarlyStopping 클래스 정의
class EarlyStopping:
    def __init__(self, patience=3, verbose=False, delta=0):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.inf
        self.delta = delta

    def __call__(self, val_loss, model, path):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model, path)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model, path)
            self.counter = 0

    def save_checkpoint(self, val_loss, model, path):
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), os.path.join(path, 'checkpoint_ES.pth'))
        self.val_loss_min = val_loss



# Dynamic learning rate adjustment function
def adjust_learning_rate(optimizer, epoch, args):
    if args.lradj == 'type1':
        lr_adjust = {epoch: args.learning_rate * (0.5 ** ((epoch - 1) // 1))}
    elif args.lradj == 'type2':
        lr_adjust = {
            2: 5e-5, 4: 1e-5, 6: 5e-6, 8: 1e-6,
            10: 5e-7, 15: 1e-7, 20: 5e-8
        }
    elif args.lradj == '3':
        lr_adjust = {epoch: args.learning_rate if epoch < 10 else args.learning_rate*0.1}
    elif args.lradj == '4':
        lr_adjust = {epoch: args.learning_rate if epoch < 15 else args.learning_rate*0.1}
    elif args.lradj == '5':
        lr_adjust = {epoch: args.learning_rate if epoch < 25 else args.learning_rate*0.1}
    elif args.lradj == '6':
        lr_adjust = {epoch: args.learning_rate if epoch < 5 else args.learning_rate*0.1}  

    if epoch in lr_adjust.keys():
        lr = lr_adjust[epoch]
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr
        print(f'Updating learning rate to {lr}')


# Training loop using DataLoader
def train_model(model, dataloader, val_dataloader, optimizer, loss_function, epochs, pred_len, args):
    early_stopping = EarlyStopping(patience=args.patience, verbose=True)
    for epoch in range(epochs):
        total_loss = 0
        model.train()
        
        # Adjust learning rate
        adjust_learning_rate(optimizer, epoch, args)
        
        for batch_sequences, batch_labels in dataloader:
            optimizer.zero_grad()

            # Forward pass
            output = model(batch_sequences)
            
            # Focus on the prediction horizon for loss calculation
            # Ensure output and batch_labels have the same length (pred_len)
            output = output[:, -pred_len:, :]  # Slice to only use the prediction length
            batch_labels = batch_labels[:, -pred_len:, :]  # Adjust labels to match
            
            # Ensure shapes match before calculating loss
            assert output.shape == batch_labels.shape, f"Output shape {output.shape} and batch_labels shape {batch_labels.shape} must match"
            
            loss = loss_function(output, batch_labels)  # Compute loss
            
            loss.backward()  # Backward pass
            optimizer.step()  # Update weights

            total_loss += loss.item()

        # Validation step
        val_loss = 0
        model.eval()
        with torch.no_grad():
            for val_batch_sequences, val_batch_labels in val_dataloader:
                val_output = model(val_batch_sequences)
                val_output = val_output[:, -args.pred_len:, :]
                val_batch_labels = val_batch_labels[:, -args.pred_len:, :]
                
                val_loss += loss_function(val_output, val_batch_labels).item()
        
        val_loss /= len(val_dataloader)

        # Log epoch loss
        print(f'Epoch {epoch} | Train Loss: {total_loss / len(dataloader)} | Val Loss: {val_loss}')
        
        # Early Stopping check
        early_stopping(val_loss, model, args.save_path)
        if early_stopping.early_stop:
            print("Early stopping triggered. Stopping training.")
            break

# Run training
train_model(model, train_dataloader, val_dataloader, optimizer, loss_function, epochs=10, pred_len=configs.pred_len, args=configs)




Updating learning rate to 0.001
Epoch 0 | Train Loss: 0.15491088880684042 | Val Loss: 0.34808775330228464
Validation loss decreased (inf --> 0.348088).  Saving model ...
Updating learning rate to 0.0005
Epoch 1 | Train Loss: 0.1125494889823384 | Val Loss: 0.30323386065928
Validation loss decreased (0.348088 --> 0.303234).  Saving model ...
Updating learning rate to 0.00025
Epoch 2 | Train Loss: 0.10509673201433214 | Val Loss: 0.22221421142153086
Validation loss decreased (0.303234 --> 0.222214).  Saving model ...
Updating learning rate to 0.000125
Epoch 3 | Train Loss: 0.10250011893203025 | Val Loss: 0.22688442985305474
EarlyStopping counter: 1 out of 3
Updating learning rate to 6.25e-05
Epoch 4 | Train Loss: 0.10106335464926039 | Val Loss: 0.25658365933313254
EarlyStopping counter: 2 out of 3
Updating learning rate to 3.125e-05
Epoch 5 | Train Loss: 0.10002434930351914 | Val Loss: 0.22579508815847693
EarlyStopping counter: 3 out of 3
Early stopping triggered. Stopping training.


# 1st Model Training (1 Epoch)

In [20]:
import torch.optim as optim
import os


# Assuming that data is already scaled before being passed to the DataLoader
seq_len = 336
pred_len = 96  # Set pred_len to the desired prediction horizon

class Configs:
    def __init__(self, seq_len, pred_len, enc_in, individual, learning_rate, lradj, patience, save_path, attach_to_trend, attach_to_seasonal):
        self.seq_len = seq_len
        self.pred_len = pred_len  # Set pred_len appropriately
        self.enc_in = enc_in
        self.individual = individual
        self.learning_rate = learning_rate
        self.lradj = lradj
        self.patience = patience
        self.save_path = save_path
        self.attach_to_trend = attach_to_trend
        self.attach_to_seasonal = attach_to_seasonal

configs = Configs(seq_len=seq_len, pred_len=pred_len, enc_in=1, individual=False,
                   learning_rate=0.0005, lradj='type1',  patience=3, save_path="./model_ER", attach_to_trend=False , attach_to_seasonal=False) # 원하는 값으로 설정


model = Model(configs).to(device)
loss_function = nn.MSELoss()  # Mean Squared Error loss
optimizer = optim.Adam(model.parameters(), lr=configs.learning_rate)

if not os.path.exists(configs.save_path):
    os.makedirs(configs.save_path)

# Dynamic learning rate adjustment function
def adjust_learning_rate(optimizer, epoch, args):
    if args.lradj == 'type1':
        lr_adjust = {epoch: args.learning_rate * (0.5 ** ((epoch - 1) // 1))}
    elif args.lradj == 'type2':
        lr_adjust = {
            2: 5e-5, 4: 1e-5, 6: 5e-6, 8: 1e-6,
            10: 5e-7, 15: 1e-7, 20: 5e-8
        }
    elif args.lradj == '3':
        lr_adjust = {epoch: args.learning_rate if epoch < 10 else args.learning_rate*0.1}
    elif args.lradj == '4':
        lr_adjust = {epoch: args.learning_rate if epoch < 15 else args.learning_rate*0.1}
    elif args.lradj == '5':
        lr_adjust = {epoch: args.learning_rate if epoch < 25 else args.learning_rate*0.1}
    elif args.lradj == '6':
        lr_adjust = {epoch: args.learning_rate if epoch < 5 else args.learning_rate*0.1}  

    if epoch in lr_adjust.keys():
        lr = lr_adjust[epoch]
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr
        print(f'Updating learning rate to {lr}')


# Training loop using DataLoader
def train_model(model, dataloader, val_dataloader, optimizer, loss_function, epochs, pred_len, args):
    # early_stopping = EarlyStopping(patience=args.patience, verbose=True)
    for epoch in range(epochs):

        if epoch == 1:
            torch.save(model.state_dict(), os.path.join(args.save_path, 'checkpoint_1.pth'))
            print("Model saved at epoch 1")

        total_loss = 0
        model.train()
        
        # Adjust learning rate
        adjust_learning_rate(optimizer, epoch, args)
        
        for batch_sequences, batch_labels in dataloader:
            optimizer.zero_grad()

            # Forward pass
            output = model(batch_sequences)
            
            # Focus on the prediction horizon for loss calculation
            # Ensure output and batch_labels have the same length (pred_len)
            output = output[:, -pred_len:, :]  # Slice to only use the prediction length
            batch_labels = batch_labels[:, -pred_len:, :]  # Adjust labels to match
            
            # Ensure shapes match before calculating loss
            assert output.shape == batch_labels.shape, f"Output shape {output.shape} and batch_labels shape {batch_labels.shape} must match"
            
            loss = loss_function(output, batch_labels)  # Compute loss
            
            loss.backward()  # Backward pass
            optimizer.step()  # Update weights

            total_loss += loss.item()

        # Validation step
        val_loss = 0
        model.eval()
        with torch.no_grad():
            for val_batch_sequences, val_batch_labels in val_dataloader:
                val_output = model(val_batch_sequences)
                val_output = val_output[:, -args.pred_len:, :]
                val_batch_labels = val_batch_labels[:, -args.pred_len:, :]
                
                val_loss += loss_function(val_output, val_batch_labels).item()
        
        val_loss /= len(val_dataloader)

        # Log epoch loss
        print(f'Epoch {epoch} | Train Loss: {total_loss / len(dataloader)} | Val Loss: {val_loss}')

    torch.save(model.state_dict(), os.path.join(args.save_path, f'checkpoint_{epochs}.pth'))
    print(f"Model saved at epoch {epochs}")
        
        # # Early Stopping check
        # early_stopping(val_loss, model, args.save_path)
        # if early_stopping.early_stop:
        #     print("Early stopping triggered. Stopping training.")
        #     break

# Run training
train_model(model, train_dataloader, val_dataloader, optimizer, loss_function, epochs=1, pred_len=configs.pred_len, args=configs)




Updating learning rate to 0.001
Epoch 0 | Train Loss: 0.15814622151741728 | Val Loss: 0.2708815370597655
Model saved at epoch 1


# Perform Inference and Form Predicted Value (PV) Sequences

In [ ]:
import torch.optim as optim
import os
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import torch

checkpoint_epochs = ['ES', 1]

test_1st_results = {}

for ckpt_epoch in checkpoint_epochs:
    print(f'\n[Predicting with model from epoch {ckpt_epoch}]')

    model_path = os.path.join(configs.save_path, f'checkpoint_{ckpt_epoch}.pth')
    
    model = Model(configs).to(device)
    
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    predictions = []
    actuals = []

    with torch.no_grad():
        for batch_sequence, batch_label in test_dataloader:
            batch_sequence = batch_sequence.float().to(device)
            batch_label = batch_label.float().to(device)

            output = model(batch_sequence)
            output = output[:, -configs.pred_len:, :]
            batch_label = batch_label[:, -configs.pred_len:, :]

            predictions.append(output.detach().cpu().numpy())
            actuals.append(batch_label.detach().cpu().numpy())

    predictions = np.concatenate(predictions, axis=0)  # shape: (N, pred_len, 1)
    actuals = np.concatenate(actuals, axis=0)

    predictions_flatten = predictions.flatten()
    actuals_flatten = actuals.flatten()


    mse = mean_squared_error(actuals_flatten, predictions_flatten)
    mae = mean_absolute_error(actuals_flatten, predictions_flatten)

    print(f'Epoch {ckpt_epoch} | MSE: {mse} | MAE: {mae}')

    num_samples = len(predictions_flatten) // configs.pred_len
    test_pv = torch.tensor(
        predictions_flatten.reshape(num_samples, configs.pred_len, 1),
        dtype=torch.float32
    )
    test_1st_results[ckpt_epoch] = {
        f'mse_test_E{ckpt_epoch}': mse,
        f'mae_test_E{ckpt_epoch}': mae,
        f'test_pv_E{ckpt_epoch}': test_pv
    }




[Predicting with model from epoch ES]
Epoch ES | MSE: 0.10531558096408844 | MAE: 0.24949122965335846

[Predicting with model from epoch 1]
Epoch 1 | MSE: 0.129582479596138 | MAE: 0.2797997295856476


In [22]:
test_1st_results.keys()

dict_keys(['ES', 1])

In [ ]:
import torch.optim as optim
import os
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import torch

checkpoint_epochs = ['ES', 1]

val_1st_results = {}

for ckpt_epoch in checkpoint_epochs:
    print(f'\n[Predicting with model from epoch {ckpt_epoch}]')

    # 모델 불러오기
    model_path = os.path.join(configs.save_path, f'checkpoint_{ckpt_epoch}.pth')
    
    model = Model(configs).to(device)
    
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    predictions = []
    actuals = []

    with torch.no_grad():
        for batch_sequence, batch_label in val_dataloader:
            batch_sequence = batch_sequence.float().to(device)
            batch_label = batch_label.float().to(device)

            output = model(batch_sequence)
            output = output[:, -configs.pred_len:, :]
            batch_label = batch_label[:, -configs.pred_len:, :]

            predictions.append(output.detach().cpu().numpy())
            actuals.append(batch_label.detach().cpu().numpy())

    predictions = np.concatenate(predictions, axis=0)  # shape: (N, pred_len, 1)
    actuals = np.concatenate(actuals, axis=0)

    predictions_flatten = predictions.flatten()
    actuals_flatten = actuals.flatten()

    mse = mean_squared_error(actuals_flatten, predictions_flatten)
    mae = mean_absolute_error(actuals_flatten, predictions_flatten)

    print(f'Epoch {ckpt_epoch} | MSE: {mse} | MAE: {mae}')

    num_samples = len(predictions_flatten) // configs.pred_len
    val_pv = torch.tensor(
        predictions_flatten.reshape(num_samples, configs.pred_len, 1),
        dtype=torch.float32
    )

    val_1st_results[ckpt_epoch] = {
        f'mse_val_E{ckpt_epoch}': mse,
        f'mae_val_E{ckpt_epoch}': mae,
        f'val_pv_E{ckpt_epoch}': val_pv
    }




[Predicting with model from epoch ES]
Epoch ES | MSE: 0.22214126586914062 | MAE: 0.3673437535762787

[Predicting with model from epoch 1]
Epoch 1 | MSE: 0.27007246017456055 | MAE: 0.4045541286468506


In [ ]:
import torch.optim as optim
import os
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import torch

# Train DataLoader for inference
train_dataloader_infer = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

checkpoint_epochs = ['ES', 1]

train_1st_results = {}

for ckpt_epoch in checkpoint_epochs:
    print(f'\n[Predicting with model from epoch {ckpt_epoch}]')

    model_path = os.path.join(configs.save_path, f'checkpoint_{ckpt_epoch}.pth')
    
    model = Model(configs).to(device)
    
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    predictions = []
    actuals = []

    with torch.no_grad():
        for batch_sequence, batch_label in train_dataloader_infer:
            batch_sequence = batch_sequence.float().to(device)
            batch_label = batch_label.float().to(device)

            output = model(batch_sequence)
            output = output[:, -configs.pred_len:, :]
            batch_label = batch_label[:, -configs.pred_len:, :]

            predictions.append(output.detach().cpu().numpy())
            actuals.append(batch_label.detach().cpu().numpy())

    predictions = np.concatenate(predictions, axis=0)  
    actuals = np.concatenate(actuals, axis=0)

    predictions_flatten = predictions.flatten()
    actuals_flatten = actuals.flatten()

    mse = mean_squared_error(actuals_flatten, predictions_flatten)
    mae = mean_absolute_error(actuals_flatten, predictions_flatten)

    print(f'Epoch {ckpt_epoch} | MSE: {mse} | MAE: {mae}')

    num_samples = len(predictions_flatten) // configs.pred_len
    train_pv = torch.tensor(
        predictions_flatten.reshape(num_samples, configs.pred_len, 1),
        dtype=torch.float32
    )

    train_1st_results[ckpt_epoch] = {
        f'mse_train_E{ckpt_epoch}': mse,
        f'mae_train_E{ckpt_epoch}': mae,
        f'train_pv_E{ckpt_epoch}': train_pv
    }




[Predicting with model from epoch ES]
Epoch ES | MSE: 0.10161511600017548 | MAE: 0.22290582954883575

[Predicting with model from epoch 1]
Epoch 1 | MSE: 0.11351648718118668 | MAE: 0.24444888532161713


In [25]:
print("train_sequences shape: ",train_sequences.shape)
print("val_sequences shape: ",val_sequences.shape)
print("test_sequences shape: ",test_sequences.shape)

print('---------------------------------------------')

print("train_pv shape: ",train_pv.shape)
print("val_pv shape: ",val_pv.shape)
print("test_pv shape: ",test_pv.shape)


print('---------------------------------------------')

print("train_labels shape: ",train_labels.shape)
print("val_labels shape: ",val_labels.shape)
print("test_labels shape: ",test_labels.shape)


train_sequences shape:  torch.Size([4880, 336, 1])
val_sequences shape:  torch.Size([665, 336, 1])
test_sequences shape:  torch.Size([1422, 336, 1])
---------------------------------------------
train_pv shape:  torch.Size([4880, 96, 1])
val_pv shape:  torch.Size([665, 96, 1])
test_pv shape:  torch.Size([1422, 96, 1])
---------------------------------------------
train_labels shape:  torch.Size([4880, 96, 1])
val_labels shape:  torch.Size([665, 96, 1])
test_labels shape:  torch.Size([1422, 96, 1])


# Segment Generation Function (sliding window-based)

In [ ]:
def create_sliding_segments(predictions, labels, segment_length, stride=1):
    B, T, C = predictions.shape
    num_segments = (T - segment_length) // stride + 1

    pred_segments = []
    label_segments = []

    for i in range(0, T - segment_length + 1, stride):
        pred_seg = predictions[:, i:i+segment_length, :]  
        label_seg = labels[:, i:i+segment_length, :]     
        pred_segments.append(pred_seg.unsqueeze(1))       
        label_segments.append(label_seg.unsqueeze(1))

    pred_segments = torch.cat(pred_segments, dim=1)       
    label_segments = torch.cat(label_segments, dim=1)     

    return pred_segments, label_segments

# Segmentation for PV Sequence

In [ ]:
configs.pred_len 

96

In [ ]:
segment_len = configs.pred_len // 3
stride = 1  

train_segments_by_epoch = {}
val_segments_by_epoch = {}
test_segments_by_epoch = {}

for epoch in checkpoint_epochs:
    train_pv = train_1st_results[epoch][f'train_pv_E{epoch}']  
    val_pv = val_1st_results[epoch][f'val_pv_E{epoch}']        
    test_pv = test_1st_results[epoch][f'test_pv_E{epoch}']     

    train_pred_segments, train_label_segments = create_sliding_segments(train_pv, train_labels, segment_len,stride)
    val_pred_segments, val_label_segments = create_sliding_segments(val_pv, val_labels, segment_len, stride)
    test_pred_segments, test_label_segments = create_sliding_segments(test_pv, test_labels, segment_len, stride)

    train_segments_by_epoch[epoch] = {
        f'train_pred_segments_E{epoch}': train_pred_segments,  
        f'train_label_segments_E{epoch}': train_label_segments
    }

    val_segments_by_epoch[epoch] = {
        f'val_pred_segments_E{epoch}': val_pred_segments,
        f'val_label_segments_E{epoch}': val_label_segments
    }

    test_segments_by_epoch[epoch] = {
        f'test_pred_segments_E{epoch}': test_pred_segments,
        f'test_label_segments_E{epoch}': test_label_segments
    }

    print(f"[Epoch {epoch}] Train seg shape: {train_pred_segments.shape}, Val: {val_pred_segments.shape}, Test: {test_pred_segments.shape}")


[Epoch ES] Train seg shape: torch.Size([4880, 65, 32, 1]), Val: torch.Size([665, 65, 32, 1]), Test: torch.Size([1422, 65, 32, 1])
[Epoch 1] Train seg shape: torch.Size([4880, 65, 32, 1]), Val: torch.Size([665, 65, 32, 1]), Test: torch.Size([1422, 65, 32, 1])


## Save 1st Performance Metrics, PV, and Segment

In [ ]:
save_dir = './1st_results_ER'  
os.makedirs(save_dir, exist_ok=True)

torch.save(train_1st_results, os.path.join(save_dir, 'train_1st_results.pt'))
torch.save(val_1st_results, os.path.join(save_dir, 'val_1st_results.pt'))
torch.save(test_1st_results, os.path.join(save_dir, 'test_1st_results.pt'))

torch.save(train_segments_by_epoch, os.path.join(save_dir, 'train_segments_by_epoch.pt'))
torch.save(val_segments_by_epoch, os.path.join(save_dir, 'val_segments_by_epoch.pt'))
torch.save(test_segments_by_epoch, os.path.join(save_dir, 'test_segments_by_epoch.pt'))

print("✅ All dictionaries saved successfully in ./saved_results")


✅ All dictionaries saved successfully in ./saved_results


# 2nd Model Training

In [30]:
train_segments_by_epoch = torch.load('./1st_results_ER/train_segments_by_epoch.pt', weights_only=False)
val_segments_by_epoch = torch.load('./1st_results_ER/val_segments_by_epoch.pt', weights_only=False)
test_segments_by_epoch = torch.load('./1st_results_ER/test_segments_by_epoch.pt', weights_only=False)

In [31]:

print(train_segments_by_epoch.keys())
print(train_segments_by_epoch['ES'].keys())
print(train_segments_by_epoch[1].keys())

dict_keys(['ES', 1])
dict_keys(['train_pred_segments_EES', 'train_label_segments_EES'])
dict_keys(['train_pred_segments_E1', 'train_label_segments_E1'])


In [ ]:

train_pred_segments_ES = train_segments_by_epoch['ES']['train_pred_segments_EES']  
train_label_segments_ES = train_segments_by_epoch['ES']['train_label_segments_EES']
val_pred_segments_ES = val_segments_by_epoch['ES']['val_pred_segments_EES']  
val_label_segments_ES = val_segments_by_epoch['ES']['val_label_segments_EES']
test_pred_segments_ES = test_segments_by_epoch['ES']['test_pred_segments_EES']  
test_label_segments_ES = test_segments_by_epoch['ES']['test_label_segments_EES']


train_pred_segments_E1 = train_segments_by_epoch[1]['train_pred_segments_E1']  
train_label_segments_E1 = train_segments_by_epoch[1]['train_label_segments_E1']
val_pred_segments_E1 = val_segments_by_epoch[1]['val_pred_segments_E1']  
val_label_segments_E1 = val_segments_by_epoch[1]['val_label_segments_E1']
test_pred_segments_E1 = test_segments_by_epoch[1]['test_pred_segments_E1']  
test_label_segments_E1 = test_segments_by_epoch[1]['test_label_segments_E1']



## ES->1E

In [ ]:
import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np, os
from pathlib import Path
import time


epo = 'ES'

class Configs:
    def __init__(self, seq_len, pred_len, enc_in, individual,
                 learning_rate, lradj, patience,
                 save_path, attach_to_trend, attach_to_seasonal):
        self.seq_len  = seq_len
        self.pred_len = pred_len
        self.enc_in   = enc_in          # 채널 수
        self.individual = individual
        self.learning_rate = learning_rate
        self.lradj = lradj
        self.patience = patience
        self.save_path = save_path
        self.attach_to_trend    = attach_to_trend
        self.attach_to_seasonal = attach_to_seasonal


base_cfg = Configs(
    seq_len          = 336 + pred_len//3,
    pred_len         = 96,
    enc_in           = 1,
    individual       = True,
    learning_rate    = 0.0005,
    lradj            = 'type1',
    patience         = 3,
    save_path        = f"./model2_ER_E{epo}_1E",
    attach_to_trend  = True,
    attach_to_seasonal = True
)
Path(base_cfg.save_path).mkdir(parents=True, exist_ok=True)

loss_fn = nn.MSELoss()

for seg in range(len(train_pred_segments_ES[0])):
    print(f"\n=========== Segment {seg} ===========")
    start_time = time.time()

    torch.cuda.reset_peak_memory_stats(device)
    mem_start = torch.cuda.memory_allocated(device)

    tr_pv = train_pred_segments_ES[:, seg, :, :]   # (N,P,C)
    va_pv = val_pred_segments_ES[:,  seg, :, :]

    tr_ds = TensorDataset(train_sequences, train_labels, tr_pv)
    va_ds = TensorDataset(val_sequences,   val_labels,   va_pv)
    tr_dl = DataLoader(tr_ds, batch_size=8, shuffle=True)
    va_dl = DataLoader(va_ds, batch_size=8, shuffle=False)

    model = Model(base_cfg).to(device)
    C     = model.channels                                 

    opts = []
    for c in range(C):
        params = list(model.Linear_Seasonal[c].parameters()) + \
                 list(model.Linear_Trend[c].parameters())
        opts.append(optim.Adam(params, lr=base_cfg.learning_rate))
        path = Path(base_cfg.save_path) / f"seg{seg}" / f"ch{c}"
        path.mkdir(parents=True, exist_ok=True)
        
    active = set(range(C))            

  
    for epoch in range(1, 2):
        if not active: break       

        model.train()
        for c in active:                             
            if base_cfg.lradj=='type1':
                lr = base_cfg.learning_rate * 0.5**((epoch-1)//1)
                for g in opts[c].param_groups: g['lr'] = lr

        printed_tensor_info = False  

        for xb, yb, pv in tr_dl:
            xb, yb, pv = xb.float().to(device), yb.float().to(device), pv.float().to(device)

            if not printed_tensor_info:
                xb_size = xb.numel() * xb.element_size() / 1024**2
                yb_size = yb.numel() * yb.element_size() / 1024**2
                pv_size = pv.numel() * pv.element_size() / 1024**2

                num_batches = len(tr_dl)

                total_xb_MB = num_batches * xb_size
                total_yb_MB = num_batches * yb_size
                total_pv_MB = num_batches * pv_size

                print(f"[Segment {seg}] Batch Size Info:")
                print(f"  xb per batch = {xb_size:.4f} MB, total = {total_xb_MB:.4f} MB")
                print(f"  yb per batch = {yb_size:.4f} MB, total = {total_yb_MB:.4f} MB")
                print(f"  pv per batch = {pv_size:.4f} MB, total = {total_pv_MB:.4f} MB")

                printed_tensor_info = True

            out = model(xb, pv)                

            loss_list = []
            for c in active:
                pred = out[:, -base_cfg.pred_len:, c]
                tgt  = yb[:, -base_cfg.pred_len:, c]
                loss_list.append(loss_fn(pred, tgt))

            total = torch.stack(loss_list).sum()
            for opt in opts:
                if opt: opt.zero_grad()
            total.backward()
            for c in active:
                opts[c].step()
    elapsed = time.time() - start_time
    mem_end = torch.cuda.memory_allocated(device)
    peak_mem = torch.cuda.max_memory_allocated(device)

    print(f"[Segment {seg}] Training Time = {elapsed:.2f} sec")
    print(f"[Segment {seg}] GPU Memory Start = {mem_start/1024**2:.2f} MB")
    print(f"[Segment {seg}] GPU Memory End   = {mem_end/1024**2:.2f} MB")
    print(f"[Segment {seg}] GPU Peak Memory  = {peak_mem/1024**2:.2f} MB")

    print(f"\n============= Done =============")

    torch.save(model.state_dict(),
                Path(base_cfg.save_path) / f"seg{seg}" / f"ch{c}" / "checkpoint.pth")
    del model, opts; torch.cuda.empty_cache()




=========== Segment 0 ===========
[Segment 0] Batch Size Info:
  xb per batch = 0.0103 MB, total = 6.2549 MB
  yb per batch = 0.0029 MB, total = 1.7871 MB
  pv per batch = 0.0010 MB, total = 0.5957 MB
[Segment 0] Training Time = 3.02 sec
[Segment 0] GPU Memory Start = 252.97 MB
[Segment 0] GPU Memory End   = 197.83 MB
[Segment 0] GPU Peak Memory  = 252.97 MB

============= Done =============

=========== Segment 1 ===========
[Segment 1] Batch Size Info:
  xb per batch = 0.0103 MB, total = 6.2549 MB
  yb per batch = 0.0029 MB, total = 1.7871 MB
  pv per batch = 0.0010 MB, total = 0.5957 MB
[Segment 1] Training Time = 3.00 sec
[Segment 1] GPU Memory Start = 197.83 MB
[Segment 1] GPU Memory End   = 198.10 MB
[Segment 1] GPU Peak Memory  = 199.14 MB

============= Done =============

=========== Segment 2 ===========
[Segment 2] Batch Size Info:
  xb per batch = 0.0103 MB, total = 6.2549 MB
  yb per batch = 0.0029 MB, total = 1.7871 MB
  pv per batch = 0.0010 MB, total = 0.5957 MB
[Segme

## 1E->1E

In [ ]:
import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np, os
from pathlib import Path

epo = 1

class Configs:
    def __init__(self, seq_len, pred_len, enc_in, individual,
                 learning_rate, lradj, patience,
                 save_path, attach_to_trend, attach_to_seasonal):
        self.seq_len  = seq_len
        self.pred_len = pred_len
        self.enc_in   = enc_in         
        self.individual = individual
        self.learning_rate = learning_rate
        self.lradj = lradj
        self.patience = patience
        self.save_path = save_path
        self.attach_to_trend    = attach_to_trend
        self.attach_to_seasonal = attach_to_seasonal

base_cfg = Configs(
    seq_len          = 336 + pred_len//3,
    pred_len         = 96,
    enc_in           = 1,
    individual       = True,
    learning_rate    = 0.0005,
    lradj            = 'type1',
    patience         = 3,
    save_path        = f"./model2_ER_E{epo}_1E/",
    attach_to_trend  = True,
    attach_to_seasonal = True
)
Path(base_cfg.save_path).mkdir(parents=True, exist_ok=True)

loss_fn = nn.MSELoss()

for seg in range(len(train_pred_segments_E1[0])):
    print(f"\n=========== Segment {seg} ===========")
    
    start_time = time.time()

    torch.cuda.reset_peak_memory_stats(device)
    mem_start = torch.cuda.memory_allocated(device)

    tr_pv = train_pred_segments_E1[:, seg, :, :]  
    va_pv = val_pred_segments_E1[:,  seg, :, :]

    tr_ds = TensorDataset(train_sequences, train_labels, tr_pv)
    va_ds = TensorDataset(val_sequences,   val_labels,   va_pv)
    tr_dl = DataLoader(tr_ds, batch_size=8, shuffle=True)
    va_dl = DataLoader(va_ds, batch_size=8, shuffle=False)

    model = Model(base_cfg).to(device)
    C     = model.channels                                  

    opts = []
    for c in range(C):
        params = list(model.Linear_Seasonal[c].parameters()) + \
                 list(model.Linear_Trend[c].parameters())
        opts.append(optim.Adam(params, lr=base_cfg.learning_rate))
        path = Path(base_cfg.save_path) / f"seg{seg}" / f"ch{c}"
        path.mkdir(parents=True, exist_ok=True)
        
    active = set(range(C))          

    for epoch in range(1, 2):
        if not active: break         

        model.train()
        for c in active:                         
            if base_cfg.lradj=='type1':
                lr = base_cfg.learning_rate * 0.5**((epoch-1)//1)
                for g in opts[c].param_groups: g['lr'] = lr

        printed_tensor_info = False
        
        for xb, yb, pv in tr_dl:
            xb, yb, pv = xb.float().to(device), yb.float().to(device), pv.float().to(device)

            if not printed_tensor_info:
                xb_size = xb.numel() * xb.element_size() / 1024**2
                yb_size = yb.numel() * yb.element_size() / 1024**2
                pv_size = pv.numel() * pv.element_size() / 1024**2

                num_batches = len(tr_dl)

                total_xb_MB = num_batches * xb_size
                total_yb_MB = num_batches * yb_size
                total_pv_MB = num_batches * pv_size
                # ─────────────────────────────


                print(f"[Segment {seg}] Batch Size Info:")
                print(f"  xb per batch = {xb_size:.4f} MB, total = {total_xb_MB:.4f} MB")
                print(f"  yb per batch = {yb_size:.4f} MB, total = {total_yb_MB:.4f} MB")
                print(f"  pv per batch = {pv_size:.4f} MB, total = {total_pv_MB:.4f} MB")

                printed_tensor_info = True


            out = model(xb, pv)                   

            loss_list = []
            for c in active:
                pred = out[:, -base_cfg.pred_len:, c]
                tgt  = yb[:, -base_cfg.pred_len:, c]
                loss_list.append(loss_fn(pred, tgt))

            total = torch.stack(loss_list).sum()
            for opt in opts:
                if opt: opt.zero_grad()
            total.backward()
            for c in active:
                opts[c].step()

    elapsed = time.time() - start_time
    mem_end = torch.cuda.memory_allocated(device)
    peak_mem = torch.cuda.max_memory_allocated(device)


    print(f"[Segment {seg}] Training Time = {elapsed:.2f} sec")
    print(f"[Segment {seg}] GPU Memory Start = {mem_start/1024**2:.2f} MB")
    print(f"[Segment {seg}] GPU Memory End   = {mem_end/1024**2:.2f} MB")
    print(f"[Segment {seg}] GPU Peak Memory  = {peak_mem/1024**2:.2f} MB")

    print(f"\n============= Done =============")


    torch.save(model.state_dict(),
                Path(base_cfg.save_path) / f"seg{seg}" / f"ch{c}" / "checkpoint.pth")
    del model, opts; torch.cuda.empty_cache()



=========== Segment 0 ===========
[Segment 0] Batch Size Info:
  xb per batch = 0.0103 MB, total = 6.2549 MB
  yb per batch = 0.0029 MB, total = 1.7871 MB
  pv per batch = 0.0010 MB, total = 0.5957 MB
[Segment 0] Training Time = 1.69 sec
[Segment 0] GPU Memory Start = 198.10 MB
[Segment 0] GPU Memory End   = 198.10 MB
[Segment 0] GPU Peak Memory  = 199.41 MB

============= Done =============

=========== Segment 1 ===========
[Segment 1] Batch Size Info:
  xb per batch = 0.0103 MB, total = 6.2549 MB
  yb per batch = 0.0029 MB, total = 1.7871 MB
  pv per batch = 0.0010 MB, total = 0.5957 MB
[Segment 1] Training Time = 1.44 sec
[Segment 1] GPU Memory Start = 198.10 MB
[Segment 1] GPU Memory End   = 198.37 MB
[Segment 1] GPU Peak Memory  = 199.41 MB

============= Done =============

=========== Segment 2 ===========
[Segment 2] Batch Size Info:
  xb per batch = 0.0103 MB, total = 6.2549 MB
  yb per batch = 0.0029 MB, total = 1.7871 MB
  pv per batch = 0.0010 MB, total = 0.5957 MB
[Segme

# 2nd Inference for Validation Set

In [35]:
train_segments_by_epoch = torch.load('./1st_results_ER/train_segments_by_epoch.pt', weights_only=False)
val_segments_by_epoch = torch.load('./1st_results_ER/val_segments_by_epoch.pt', weights_only=False)
test_segments_by_epoch = torch.load('./1st_results_ER/test_segments_by_epoch.pt', weights_only=False)

In [ ]:
train_pred_segments_ES = train_segments_by_epoch['ES']['train_pred_segments_EES']  
train_label_segments_ES = train_segments_by_epoch['ES']['train_label_segments_EES']
val_pred_segments_ES = val_segments_by_epoch['ES']['val_pred_segments_EES']  
val_label_segments_ES = val_segments_by_epoch['ES']['val_label_segments_EES']
test_pred_segments_ES = test_segments_by_epoch['ES']['test_pred_segments_EES']  
test_label_segments_ES = test_segments_by_epoch['ES']['test_label_segments_EES']


train_pred_segments_E1 = train_segments_by_epoch[1]['train_pred_segments_E1']  
train_label_segments_E1 = train_segments_by_epoch[1]['train_label_segments_E1']
val_pred_segments_E1 = val_segments_by_epoch[1]['val_pred_segments_E1']  
val_label_segments_E1 = val_segments_by_epoch[1]['val_label_segments_E1']
test_pred_segments_E1 = test_segments_by_epoch[1]['test_pred_segments_E1']  
test_label_segments_E1 = test_segments_by_epoch[1]['test_label_segments_E1']


## ES->1E

### Configs

In [ ]:
import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np, os
from pathlib import Path

class Configs:
    def __init__(self, seq_len, pred_len, enc_in, individual,
                 learning_rate, lradj, patience,
                 save_path, attach_to_trend, attach_to_seasonal):
        self.seq_len  = seq_len
        self.pred_len = pred_len
        self.enc_in   = enc_in          
        self.individual = individual
        self.learning_rate = learning_rate
        self.lradj = lradj
        self.patience = patience
        self.save_path = save_path
        self.attach_to_trend    = attach_to_trend
        self.attach_to_seasonal = attach_to_seasonal

base_cfg = Configs(
    pred_len         = 96,
    seq_len          = 336 + pred_len//3,
    enc_in           = 1,
    individual       = True,
    learning_rate    = 0.0005,
    lradj            = 'type1',
    patience         = 3,
    save_path        = "./model2_ER_EES_1E",
    attach_to_trend  = True,
    attach_to_seasonal = True
)
Path(base_cfg.save_path).mkdir(parents=True, exist_ok=True)

### Inference

In [ ]:
import torch, numpy as np
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import r2_score
from pathlib import Path


C        = 1                                
P        = base_cfg.pred_len                
seg_cnt  = 65
batch_sz = 8

results2_ER_ES_val_1E = {}  

for seg in range(seg_cnt):
    print(f"\n―――― Inference  |  Segment {seg} ――――")

    pv = val_pred_segments_ES[:, seg, :, :]         
    ds = TensorDataset(val_sequences, val_labels, pv) 
    dl = DataLoader(ds, batch_size=batch_sz, shuffle=False)

    pred_ch, true_ch, metrics = [], [], {}

    for ch in range(C):
        ckpt = Path(base_cfg.save_path) / f"seg{seg}" / f"ch{ch}" / "checkpoint.pth"
        if not ckpt.exists():
            print(f"[Warn] seg{seg}-ch{ch} ckpt 없음 → skip");  continue

        model = Model(base_cfg).to(device)
        model.load_state_dict(torch.load(ckpt, map_location=device))
        model.eval()
        preds, trues = [], []
        with torch.no_grad():
            for xb, yb, pv_batch in dl:                
                xb  = xb.float().to(device)
                yb  = yb.float()      
                pv_ = pv_batch.float().to(device)

                out = model(xb, pv_)                     
                preds.append(out[:, -P:, ch].cpu())
                trues.append(yb[:, -P:, ch])

        preds = torch.cat(preds, 0)     
        trues = torch.cat(trues, 0)
        pred_ch.append(preds)
        true_ch.append(trues)

        p, t = preds.reshape(-1).cpu().numpy(), trues.reshape(-1).cpu().numpy()
        mse  = np.mean((p - t)**2)
        mae  = np.mean(np.abs(p - t))
        metrics[ch] = (mse, mae)

        print(f"  seg{seg}-ch{ch}  MSE:{mse}  MAE:{mae}")

        del model; torch.cuda.empty_cache()
    
    pred_all = torch.stack(pred_ch,  dim=2)   
    true_all = torch.stack(true_ch,  dim=2)

    mse_avg = np.mean([m for m,_ in metrics.values()])
    mae_avg = np.mean([a for _,a in metrics.values()])
    print(f"▶ seg{seg}  mean  MSE:{mse_avg}  MAE:{mae_avg}")

    results2_ER_ES_val_1E[seg] = {
        "pred2_ES_val_1E"    : pred_all,   
        "true2_ES_val_1E"    : true_all,   
        "metrics2_ES_val_1E" : metrics     
    }

seg10_ch0_mse = results2_ER_ES_val_1E[10]["metrics2_ES_val_1E"][0][0]
print("\nseg10 ch0 MSE :", seg10_ch0_mse)



―――― Inference  |  Segment 0 ――――
  seg0-ch0  MSE:0.2852832078933716  MAE:0.410333514213562
▶ seg0  mean  MSE:0.2852832078933716  MAE:0.410333514213562

―――― Inference  |  Segment 1 ――――
  seg1-ch0  MSE:0.23253001272678375  MAE:0.3758050501346588
▶ seg1  mean  MSE:0.23253001272678375  MAE:0.3758050501346588

―――― Inference  |  Segment 2 ――――
  seg2-ch0  MSE:0.21294039487838745  MAE:0.35975152254104614
▶ seg2  mean  MSE:0.21294039487838745  MAE:0.35975152254104614

―――― Inference  |  Segment 3 ――――
  seg3-ch0  MSE:0.21830131113529205  MAE:0.3584003448486328
▶ seg3  mean  MSE:0.21830131113529205  MAE:0.3584003448486328

―――― Inference  |  Segment 4 ――――
  seg4-ch0  MSE:0.23349851369857788  MAE:0.374158650636673
▶ seg4  mean  MSE:0.23349851369857788  MAE:0.374158650636673

―――― Inference  |  Segment 5 ――――
  seg5-ch0  MSE:0.22552432119846344  MAE:0.37359511852264404
▶ seg5  mean  MSE:0.22552432119846344  MAE:0.37359511852264404

―――― Inference  |  Segment 6 ――――
  seg6-ch0  MSE:0.2246253

## 1E->1E

### Configs

In [ ]:
import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np, os
from pathlib import Path


class Configs:
    def __init__(self, seq_len, pred_len, enc_in, individual,
                 learning_rate, lradj, patience,
                 save_path, attach_to_trend, attach_to_seasonal):
        self.seq_len  = seq_len
        self.pred_len = pred_len
        self.enc_in   = enc_in         
        self.individual = individual
        self.learning_rate = learning_rate
        self.lradj = lradj
        self.patience = patience
        self.save_path = save_path
        self.attach_to_trend    = attach_to_trend
        self.attach_to_seasonal = attach_to_seasonal

base_cfg = Configs(
    pred_len         = 96,
    seq_len          = 336 + pred_len//3,
    enc_in           = 1,
    individual       = True,
    learning_rate    = 0.0005,
    lradj            = 'type1',
    patience         = 3,
    save_path        = "./model2_ER_E1_1E",
    attach_to_trend  = True,
    attach_to_seasonal = True
)
Path(base_cfg.save_path).mkdir(parents=True, exist_ok=True)

### Inference

In [ ]:
import torch, numpy as np
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import r2_score
from pathlib import Path


C        = 1                               
P        = base_cfg.pred_len                
seg_cnt  = 65
batch_sz = 8

results2_ER_E1_val_1E = {}  

for seg in range(seg_cnt):
    print(f"\n―――― Inference  |  Segment {seg} ――――")


    pv = val_pred_segments_E1[:, seg, :, :]        
    ds = TensorDataset(val_sequences, val_labels, pv) 
    dl = DataLoader(ds, batch_size=batch_sz, shuffle=False)

    pred_ch, true_ch, metrics = [], [], {}

    for ch in range(C):
        ckpt = Path(base_cfg.save_path) / f"seg{seg}" / f"ch{ch}" / "checkpoint.pth"
        if not ckpt.exists():
            print(f"[Warn] seg{seg}-ch{ch} ckpt 없음 → skip");  continue

        model = Model(base_cfg).to(device)
        model.load_state_dict(torch.load(ckpt, map_location=device))
        model.eval()

        preds, trues = [], []
        with torch.no_grad():
            for xb, yb, pv_batch in dl:                
                xb  = xb.float().to(device)
                yb  = yb.float()        
                pv_ = pv_batch.float().to(device)

                out = model(xb, pv_)                    
                preds.append(out[:, -P:, ch].cpu())
                trues.append(yb[:, -P:, ch])

        preds = torch.cat(preds, 0)     
        trues = torch.cat(trues, 0)
        pred_ch.append(preds)
        true_ch.append(trues)

        p, t = preds.reshape(-1).cpu().numpy(), trues.reshape(-1).cpu().numpy()
        mse  = np.mean((p - t)**2)
        mae  = np.mean(np.abs(p - t))
        metrics[ch] = (mse, mae)

        print(f"  seg{seg}-ch{ch}  MSE:{mse}  MAE:{mae}")

        del model; torch.cuda.empty_cache()

    pred_all = torch.stack(pred_ch,  dim=2)   
    true_all = torch.stack(true_ch,  dim=2)

    mse_avg = np.mean([m for m,_ in metrics.values()])
    mae_avg = np.mean([a for _,a in metrics.values()])
    print(f"▶ seg{seg}  mean  MSE:{mse_avg}  MAE:{mae_avg}")

    results2_ER_E1_val_1E[seg] = {
        "pred2_E1_val_1E"    : pred_all,   
        "true2_E1_val_1E"    : true_all,   
        "metrics2_E1_val_1E" : metrics    
    }

seg10_ch0_mse = results2_ER_E1_val_1E[10]["metrics2_E1_val_1E"][0][0]
print("\nseg10 ch0 MSE :", seg10_ch0_mse)



―――― Inference  |  Segment 0 ――――
  seg0-ch0  MSE:0.20817522704601288  MAE:0.35231953859329224
▶ seg0  mean  MSE:0.20817522704601288  MAE:0.35231953859329224

―――― Inference  |  Segment 1 ――――
  seg1-ch0  MSE:0.24446354806423187  MAE:0.3886965811252594
▶ seg1  mean  MSE:0.24446354806423187  MAE:0.3886965811252594

―――― Inference  |  Segment 2 ――――
  seg2-ch0  MSE:0.2703588902950287  MAE:0.4081382155418396
▶ seg2  mean  MSE:0.2703588902950287  MAE:0.4081382155418396

―――― Inference  |  Segment 3 ――――
  seg3-ch0  MSE:0.2515197694301605  MAE:0.38821348547935486
▶ seg3  mean  MSE:0.2515197694301605  MAE:0.38821348547935486

―――― Inference  |  Segment 4 ――――
  seg4-ch0  MSE:0.2706158459186554  MAE:0.4185120463371277
▶ seg4  mean  MSE:0.2706158459186554  MAE:0.4185120463371277

―――― Inference  |  Segment 5 ――――
  seg5-ch0  MSE:0.22414512932300568  MAE:0.3728790879249573
▶ seg5  mean  MSE:0.22414512932300568  MAE:0.3728790879249573

―――― Inference  |  Segment 6 ――――
  seg6-ch0  MSE:0.3401491

# 2nd Inference for Test Set

## ES->1E

### Configs

In [ ]:
import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np, os
from pathlib import Path

pred_len = 96
seq_len = 336 + pred_len//3

class Configs:
    def __init__(self, seq_len, pred_len, enc_in, individual,
                 learning_rate, lradj, patience,
                 save_path, attach_to_trend, attach_to_seasonal):
        self.seq_len  = seq_len
        self.pred_len = pred_len
        self.enc_in   = enc_in          
        self.individual = individual
        self.learning_rate = learning_rate
        self.lradj = lradj
        self.patience = patience
        self.save_path = save_path
        self.attach_to_trend    = attach_to_trend
        self.attach_to_seasonal = attach_to_seasonal

base_cfg = Configs(
    pred_len         = 96,
    seq_len          = 336 + pred_len//3,
    enc_in           = 1,
    individual       = True,
    learning_rate    = 0.0005,
    lradj            = 'type1',
    patience         = 3,
    save_path        = "./model2_ER_EES_1E",
    attach_to_trend  = True,
    attach_to_seasonal = True
)
Path(base_cfg.save_path).mkdir(parents=True, exist_ok=True)

### Inference

In [ ]:
import torch, numpy as np
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import r2_score
from pathlib import Path

C        = 1                               
P        = base_cfg.pred_len              
seg_cnt  = 65
batch_sz = 8

results2_ER_ES_test_1E = {}  

for seg in range(seg_cnt):
    print(f"\n―――― Inference  |  Segment {seg} ――――")

    pv = test_pred_segments_ES[:, seg, :, :]         
    ds = TensorDataset(test_sequences, test_labels, pv) 
    dl = DataLoader(ds, batch_size=batch_sz, shuffle=False)

    pred_ch, true_ch, metrics = [], [], {}

    for ch in range(C):
        ckpt = Path(base_cfg.save_path) / f"seg{seg}" / f"ch{ch}" / "checkpoint.pth"
        if not ckpt.exists():
            print(f"[Warn] seg{seg}-ch{ch} ckpt 없음 → skip");  continue

        model = Model(base_cfg).to(device)
        model.load_state_dict(torch.load(ckpt, map_location=device))
        model.eval()

        preds, trues = [], []
        with torch.no_grad():
            for xb, yb, pv_batch in dl:               
                xb  = xb.float().to(device)
                yb  = yb.float()       
                pv_ = pv_batch.float().to(device)

                out = model(xb, pv_)                    
                preds.append(out[:, -P:, ch].cpu())
                trues.append(yb[:, -P:, ch])

        preds = torch.cat(preds, 0)   
        trues = torch.cat(trues, 0)
        pred_ch.append(preds)
        true_ch.append(trues)

        p, t = preds.reshape(-1).cpu().numpy(), trues.reshape(-1).cpu().numpy()
        mse  = np.mean((p - t)**2)
        mae  = np.mean(np.abs(p - t))
        metrics[ch] = (mse, mae)

        print(f"  seg{seg}-ch{ch}  MSE:{mse}  MAE:{mae}")

        del model; torch.cuda.empty_cache()

    pred_all = torch.stack(pred_ch,  dim=2)   
    true_all = torch.stack(true_ch,  dim=2)

    mse_avg = np.mean([m for m,_ in metrics.values()])
    mae_avg = np.mean([a for _,a in metrics.values()])
    print(f"▶ seg{seg}  mean  MSE:{mse_avg}  MAE:{mae_avg}")

    results2_ER_ES_test_1E[seg] = {
        "pred2_ES_test_1E"    : pred_all,   
        "true2_ES_test_1E"    : true_all,   
        "metrics2_ES_test_1E" : metrics    
    }

seg10_ch0_mse = results2_ER_ES_test_1E[10]["metrics2_ES_test_1E"][0][0]
print("\nseg10 ch0 MSE :", seg10_ch0_mse)



―――― Inference  |  Segment 0 ――――
  seg0-ch0  MSE:0.15913228690624237  MAE:0.3078018128871918
▶ seg0  mean  MSE:0.15913228690624237  MAE:0.3078018128871918

―――― Inference  |  Segment 1 ――――
  seg1-ch0  MSE:0.10878953337669373  MAE:0.25359046459198
▶ seg1  mean  MSE:0.10878953337669373  MAE:0.25359046459198

―――― Inference  |  Segment 2 ――――
  seg2-ch0  MSE:0.09837909042835236  MAE:0.23923060297966003
▶ seg2  mean  MSE:0.09837909042835236  MAE:0.23923060297966003

―――― Inference  |  Segment 3 ――――
  seg3-ch0  MSE:0.1339806765317917  MAE:0.27912992238998413
▶ seg3  mean  MSE:0.1339806765317917  MAE:0.27912992238998413

―――― Inference  |  Segment 4 ――――
  seg4-ch0  MSE:0.11549591273069382  MAE:0.25948667526245117
▶ seg4  mean  MSE:0.11549591273069382  MAE:0.25948667526245117

―――― Inference  |  Segment 5 ――――
  seg5-ch0  MSE:0.11148955672979355  MAE:0.2614896893501282
▶ seg5  mean  MSE:0.11148955672979355  MAE:0.2614896893501282

―――― Inference  |  Segment 6 ――――
  seg6-ch0  MSE:0.11074

## 1E->1E

### Configs

In [ ]:
import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np, os
from pathlib import Path

pred_len = 96
seq_len = 336 + pred_len//3

class Configs:
    def __init__(self, seq_len, pred_len, enc_in, individual,
                 learning_rate, lradj, patience,
                 save_path, attach_to_trend, attach_to_seasonal):
        self.seq_len  = seq_len
        self.pred_len = pred_len
        self.enc_in   = enc_in          
        self.individual = individual
        self.learning_rate = learning_rate
        self.lradj = lradj
        self.patience = patience
        self.save_path = save_path
        self.attach_to_trend    = attach_to_trend
        self.attach_to_seasonal = attach_to_seasonal

base_cfg = Configs(
    pred_len         = 96,
    seq_len          = 336 + pred_len//3,
    enc_in           = 1,
    individual       = True,
    learning_rate    = 0.0005,
    lradj            = 'type1',
    patience         = 3,
    save_path        = "./model2_ER_E1_1E",
    attach_to_trend  = True,
    attach_to_seasonal = True
)
Path(base_cfg.save_path).mkdir(parents=True, exist_ok=True)

### Inference

In [ ]:
import torch, numpy as np
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import r2_score
from pathlib import Path

C        = 1                                
P        = base_cfg.pred_len               
seg_cnt  = 65
batch_sz = 8

results2_ER_E1_test_1E = {}  

for seg in range(seg_cnt):
    print(f"\n―――― Inference  |  Segment {seg} ――――")

    pv = test_pred_segments_E1[:, seg, :, :]         
    ds = TensorDataset(test_sequences, test_labels, pv) 
    dl = DataLoader(ds, batch_size=batch_sz, shuffle=False)

    pred_ch, true_ch, metrics = [], [], {}

    for ch in range(C):
        ckpt = Path(base_cfg.save_path) / f"seg{seg}" / f"ch{ch}" / "checkpoint.pth"
        if not ckpt.exists():
            print(f"[Warn] seg{seg}-ch{ch} ckpt 없음 → skip");  continue

        model = Model(base_cfg).to(device)
        model.load_state_dict(torch.load(ckpt, map_location=device))
        model.eval()

        preds, trues = [], []
        with torch.no_grad():
            for xb, yb, pv_batch in dl:                
                xb  = xb.float().to(device)
                yb  = yb.float()        
                pv_ = pv_batch.float().to(device)

                out = model(xb, pv_)                     
                preds.append(out[:, -P:, ch].cpu())
                trues.append(yb[:, -P:, ch])

        preds = torch.cat(preds, 0)     
        trues = torch.cat(trues, 0)
        pred_ch.append(preds)
        true_ch.append(trues)

        p, t = preds.reshape(-1).cpu().numpy(), trues.reshape(-1).cpu().numpy()
        mse  = np.mean((p - t)**2)
        mae  = np.mean(np.abs(p - t))
        metrics[ch] = (mse, mae)

        print(f"  seg{seg}-ch{ch}  MSE:{mse}  MAE:{mae}")

        del model; torch.cuda.empty_cache()

    pred_all = torch.stack(pred_ch,  dim=2)   
    true_all = torch.stack(true_ch,  dim=2)

    mse_avg = np.mean([m for m,_ in metrics.values()])
    mae_avg = np.mean([a for _,a in metrics.values()])
    print(f"▶ seg{seg}  mean  MSE:{mse_avg}  MAE:{mae_avg}")

    results2_ER_E1_test_1E[seg] = {
        "pred2_E1_test_1E"    : pred_all,   
        "true2_E1_test_1E"    : true_all,   
        "metrics2_E1_test_1E" : metrics     
    }

seg10_ch0_mse = results2_ER_E1_test_1E[10]["metrics2_E1_test_1E"][0][0]
print("\nseg10 ch0 MSE :", seg10_ch0_mse)



―――― Inference  |  Segment 0 ――――
  seg0-ch0  MSE:0.12613028287887573  MAE:0.262128621339798
▶ seg0  mean  MSE:0.12613028287887573  MAE:0.262128621339798

―――― Inference  |  Segment 1 ――――
  seg1-ch0  MSE:0.13172996044158936  MAE:0.2849332392215729
▶ seg1  mean  MSE:0.13172996044158936  MAE:0.2849332392215729

―――― Inference  |  Segment 2 ――――
  seg2-ch0  MSE:0.13652381300926208  MAE:0.2903972268104553
▶ seg2  mean  MSE:0.13652381300926208  MAE:0.2903972268104553

―――― Inference  |  Segment 3 ――――
  seg3-ch0  MSE:0.12427540123462677  MAE:0.27104640007019043
▶ seg3  mean  MSE:0.12427540123462677  MAE:0.27104640007019043

―――― Inference  |  Segment 4 ――――
  seg4-ch0  MSE:0.15178582072257996  MAE:0.31373801827430725
▶ seg4  mean  MSE:0.15178582072257996  MAE:0.31373801827430725

―――― Inference  |  Segment 5 ――――
  seg5-ch0  MSE:0.10469694435596466  MAE:0.25268426537513733
▶ seg5  mean  MSE:0.10469694435596466  MAE:0.25268426537513733

―――― Inference  |  Segment 6 ――――
  seg6-ch0  MSE:0.1

# Rank Models (by 2nd validation performance)

In [45]:
results1_ER_train = torch.load("./1st_results_ER/train_1st_results.pt")
results1_ER_val = torch.load("./1st_results_ER/val_1st_results.pt")
results1_ER_test = torch.load("./1st_results_ER/test_1st_results.pt")

### ES->1E

In [ ]:

import numpy as np
from sklearn.metrics import mean_squared_error


TOP_N       = 65
SEG_CNT     = 65
CH_CNT      = 1
metric_idx  = 0          

mse_table = np.full((SEG_CNT, CH_CNT), np.inf)

for seg in range(SEG_CNT):
    if seg not in results2_ER_ES_val_1E:
        continue
    for ch in range(CH_CNT):
        if ch in results2_ER_ES_val_1E[seg]["metrics2_ES_val_1E"]:
            mse_table[seg, ch] = results2_ER_ES_val_1E[seg]["metrics2_ES_val_1E"][ch][metric_idx]

print("\n▶ Train-set  Per-Channel  Top-N  MSE\n")
top_seg_idx_ES_val_1E = {}                       # ← 여기 저장

for ch in range(CH_CNT):
    print(f"=== Channel {ch} ===")
    v_mse = results1_ER_val[1]['mse_val_E1']
    print(f"Vanilla MSE : {v_mse:.6f}\n")

    pairs = [(seg, mse_table[seg, ch]) for seg in range(SEG_CNT)
             if np.isfinite(mse_table[seg, ch])]
    top   = sorted(pairs, key=lambda x: x[1])[:TOP_N]

    top_seg_idx_ES_val_1E[ch] = [seg for seg, _ in top]   # e.g. {0:[12,5,7,…], 1:[3,8,…]}

    for rank, (seg_id, mse_val) in enumerate(top, 1):
        delta = v_mse - mse_val
        sign  = "+" if delta > 0 else "-"
        print(f"  Top {rank:2d} | seg {seg_id:02d} | MSE {mse_val:.6f} "
              f"({sign}{abs(delta):.6f})")
    print()

print("Saved top segment indices per channel:")
for ch, seg_list in top_seg_idx_ES_val_1E.items():
    print(f"Channel {ch}: {seg_list}")



▶ Train-set  Per-Channel  Top-N  MSE

=== Channel 0 ===
Vanilla MSE : 0.270072

  Top  1 | seg 59 | MSE 0.203254 (+0.066818)
  Top  2 | seg 38 | MSE 0.206371 (+0.063702)
  Top  3 | seg 23 | MSE 0.208670 (+0.061402)
  Top  4 | seg 35 | MSE 0.209118 (+0.060954)
  Top  5 | seg 18 | MSE 0.209436 (+0.060637)
  Top  6 | seg 12 | MSE 0.210278 (+0.059795)
  Top  7 | seg 11 | MSE 0.211181 (+0.058892)
  Top  8 | seg 62 | MSE 0.211232 (+0.058841)
  Top  9 | seg 02 | MSE 0.212940 (+0.057132)
  Top 10 | seg 56 | MSE 0.213183 (+0.056889)
  Top 11 | seg 60 | MSE 0.213461 (+0.056612)
  Top 12 | seg 44 | MSE 0.213985 (+0.056087)
  Top 13 | seg 49 | MSE 0.214693 (+0.055380)
  Top 14 | seg 40 | MSE 0.216501 (+0.053572)
  Top 15 | seg 55 | MSE 0.217320 (+0.052752)
  Top 16 | seg 03 | MSE 0.218301 (+0.051771)
  Top 17 | seg 16 | MSE 0.219114 (+0.050958)
  Top 18 | seg 36 | MSE 0.219375 (+0.050697)
  Top 19 | seg 14 | MSE 0.221293 (+0.048779)
  Top 20 | seg 64 | MSE 0.221330 (+0.048742)
  Top 21 | seg 32 |

### 1E->1E

In [ ]:

import numpy as np
from sklearn.metrics import mean_squared_error


TOP_N       = 65
SEG_CNT     = 65
CH_CNT      = 1
metric_idx  = 0          

mse_table = np.full((SEG_CNT, CH_CNT), np.inf)

for seg in range(SEG_CNT):
    if seg not in results2_ER_E1_val_1E:
        continue
    for ch in range(CH_CNT):
        if ch in results2_ER_E1_val_1E[seg]["metrics2_E1_val_1E"]:
            mse_table[seg, ch] = results2_ER_E1_val_1E[seg]["metrics2_E1_val_1E"][ch][metric_idx]

print("\n▶ Train-set  Per-Channel  Top-N  MSE\n")
top_seg_idx_E1_val_1E = {}                    

for ch in range(CH_CNT):
    print(f"=== Channel {ch} ===")
    v_mse = results1_ER_val[1]['mse_val_E1']
    print(f"Vanilla MSE : {v_mse:.6f}\n")

    pairs = [(seg, mse_table[seg, ch]) for seg in range(SEG_CNT)
             if np.isfinite(mse_table[seg, ch])]
    top   = sorted(pairs, key=lambda x: x[1])[:TOP_N]

    top_seg_idx_E1_val_1E[ch] = [seg for seg, _ in top]   # e.g. {0:[12,5,7,…], 1:[3,8,…]}

    for rank, (seg_id, mse_val) in enumerate(top, 1):
        delta = v_mse - mse_val
        sign  = "+" if delta > 0 else "-"
        print(f"  Top {rank:2d} | seg {seg_id:02d} | MSE {mse_val:.6f} "
              f"({sign}{abs(delta):.6f})")
    print()

print("Saved top segment indices per channel:")
for ch, seg_list in top_seg_idx_E1_val_1E.items():
    print(f"Channel {ch}: {seg_list}")



▶ Train-set  Per-Channel  Top-N  MSE

=== Channel 0 ===
Vanilla MSE : 0.270072

  Top  1 | seg 35 | MSE 0.204408 (+0.065665)
  Top  2 | seg 00 | MSE 0.208175 (+0.061897)
  Top  3 | seg 58 | MSE 0.210170 (+0.059903)
  Top  4 | seg 40 | MSE 0.212111 (+0.057962)
  Top  5 | seg 61 | MSE 0.212894 (+0.057179)
  Top  6 | seg 16 | MSE 0.215181 (+0.054892)
  Top  7 | seg 25 | MSE 0.215578 (+0.054495)
  Top  8 | seg 32 | MSE 0.216163 (+0.053909)
  Top  9 | seg 14 | MSE 0.216430 (+0.053643)
  Top 10 | seg 55 | MSE 0.218009 (+0.052063)
  Top 11 | seg 18 | MSE 0.218374 (+0.051698)
  Top 12 | seg 30 | MSE 0.219828 (+0.050244)
  Top 13 | seg 05 | MSE 0.224145 (+0.045927)
  Top 14 | seg 33 | MSE 0.225478 (+0.044595)
  Top 15 | seg 41 | MSE 0.225509 (+0.044563)
  Top 16 | seg 59 | MSE 0.227122 (+0.042950)
  Top 17 | seg 64 | MSE 0.227591 (+0.042481)
  Top 18 | seg 19 | MSE 0.228226 (+0.041847)
  Top 19 | seg 27 | MSE 0.228391 (+0.041681)
  Top 20 | seg 23 | MSE 0.228990 (+0.041083)
  Top 21 | seg 54 |

# Ensemble

## ES->1E


In [ ]:
import torch, os, numpy as np
from torch.utils.data import DataLoader, TensorDataset

TOP_K    = 65
CH_CNT   = 1
batch_sz = 8

def infer_one(model, loader, ch):
    buf = []
    model.eval()
    with torch.no_grad():
        for xb, yb, pv in loader:
            xb, pv = xb.to(device), pv.to(device)
            buf.append(model(xb, pv)[:, :, ch].cpu())   # [B,P]
    return torch.cat(buf, 0)   # [N,P]


van_mse_list, ens_mse_list, delta_mse_list, delta_mse_pct_list = [], [], [], []
van_mae_list, ens_mae_list, delta_mae_list, delta_mae_pct_list = [], [], [], []

for ch in range(CH_CNT):
    print(f"\n=== Channel {ch} ===")

    v_mse = results1_ER_test['ES']['mse_test_EES']
    v_mae = results1_ER_test['ES']['mae_test_EES']

    print(f"[Vanilla]   MSE : {v_mse:.6f} | MAE : {v_mae:.6f}")

    van_mse_list.append(v_mse)
    van_mae_list.append(v_mae)

    preds_test_arr = []
    ok_segs = []

    for seg in top_seg_idx_ES_val_1E[ch][:TOP_K]:
        ckpt = f"./model2_ER_EES_1E/seg{seg}/ch{ch}/checkpoint.pth"
        if not os.path.exists(ckpt):
            print(f"  [skip] ckpt missing : seg{seg}")
            continue

        pv_seg = test_pred_segments_ES[:, seg, :, :]
        dl     = DataLoader(TensorDataset(test_sequences, test_labels, pv_seg),
                            batch_size=batch_sz, shuffle=False)

        mdl = Model(base_cfg).to(device)
        mdl.load_state_dict(torch.load(ckpt, map_location=device))

        preds_test_arr.append(infer_one(mdl, dl, ch))   # [N,P]
        ok_segs.append(seg)

    if len(preds_test_arr) < 5:
        print("  Not enough valid models.")
        continue

    step = 5
    Ks = list(range(step, len(preds_test_arr)+1, step))

    var_list, rho_list = [], []

    for K in Ks:
        F = torch.stack(preds_test_arr[:K], dim=0)

        V = torch.var(F, dim=0).mean().item()
        var_list.append(V)

        flat = F.reshape(K, -1).numpy()
        corr = np.corrcoef(flat)
        R = np.mean(np.abs(corr[np.triu_indices(K, 1)]))
        rho_list.append(R)

    # 정규화
    V_min, V_max = min(var_list), max(var_list)
    R_min, R_max = min(rho_list), max(rho_list)

    V_norm = [(v - V_min)/(V_max - V_min + 1e-12) for v in var_list]
    R_norm = [(r - R_min)/(R_max - R_min + 1e-12) for r in rho_list]

    alpha, beta = 1, 1
    scores = [alpha*v + beta*r for v, r in zip(V_norm, R_norm)]

    best_K = Ks[int(np.argmin(scores))]

    print("  — Var / |Corr| / Score —")
    for k, v, r, s in zip(Ks, var_list, rho_list, scores):
        tag = "<- pick" if k == best_K else ""
        print(f"   K={k:2d} | Var={v:.3e} | |ρ|={r:.4f} | S={s:.4f} {tag}")

    ens_pred = torch.mean(torch.stack(preds_test_arr[:best_K]), dim=0)
    y_true   = test_labels[:, -base_cfg.pred_len:, ch].cpu()

    mse_ens = torch.mean((ens_pred - y_true)**2).item()
    mae_ens = torch.mean(torch.abs(ens_pred - y_true)).item()

    ens_mse_list.append(mse_ens)
    ens_mae_list.append(mae_ens)

    delta_mse = v_mse - mse_ens
    delta_mae = v_mae - mae_ens

    delta_mse_pct = (delta_mse / v_mse) * 100
    delta_mae_pct = (delta_mae / v_mae) * 100

    delta_mse_list.append(delta_mse)
    delta_mae_list.append(delta_mae)

    delta_mse_pct_list.append(delta_mse_pct)
    delta_mae_pct_list.append(delta_mae_pct)

    print(f"[Var-Corr]  K={best_K}")
    print(f"  MSE = {mse_ens:.6f} | Δ = {delta_mse:+.6f} | Δ% = {delta_mse_pct:+.2f}%")
    print(f"  MAE = {mae_ens:.6f} | Δ = {delta_mae:+.6f} | Δ% = {delta_mae_pct:+.2f}%")

if ens_mse_list:
    print("\n──────── Summary (Var-Corr) ───────")
    print(f"Vanilla  mean MSE : {np.mean(van_mse_list):.6f}")
    print(f"Ensemble mean MSE : {np.mean(ens_mse_list):.6f}")
    print(f"Average Δ MSE     : {np.mean(delta_mse_list):+.6f}")
    print(f"Average Δ MSE (%) : {np.mean(delta_mse_pct_list):+.2f}%\n")

    print(f"Vanilla  mean MAE : {np.mean(van_mae_list):.6f}")
    print(f"Ensemble mean MAE : {np.mean(ens_mae_list):.6f}")
    print(f"Average Δ MAE     : {np.mean(delta_mae_list):+.6f}")
    print(f"Average Δ MAE (%) : {np.mean(delta_mae_pct_list):+.2f}%")



=== Channel 0 ===
[Vanilla]   MSE : 0.105316 | MAE : 0.249491


  — Var / |Corr| / Score —
   K= 5 | Var=1.401e-02 | |ρ|=0.9841 | S=0.9244 
   K=10 | Var=1.136e-02 | |ρ|=0.9865 | S=1.0000 
   K=15 | Var=1.343e-02 | |ρ|=0.9846 | S=0.9428 
   K=20 | Var=1.421e-02 | |ρ|=0.9837 | S=0.9025 <- pick
   K=25 | Var=1.471e-02 | |ρ|=0.9835 | S=0.9054 
   K=30 | Var=1.630e-02 | |ρ|=0.9832 | S=0.9466 
   K=35 | Var=1.733e-02 | |ρ|=0.9823 | S=0.9231 
   K=40 | Var=1.811e-02 | |ρ|=0.9828 | S=0.9876 
   K=45 | Var=1.923e-02 | |ρ|=0.9814 | S=0.9348 
   K=50 | Var=2.122e-02 | |ρ|=0.9801 | S=0.9152 
   K=55 | Var=2.443e-02 | |ρ|=0.9790 | S=0.9554 
   K=60 | Var=2.858e-02 | |ρ|=0.9766 | S=0.9443 
   K=65 | Var=3.681e-02 | |ρ|=0.9730 | S=1.0000 
[Var-Corr]  K=20
  MSE = 0.101966 | Δ = +0.003349 | Δ% = +3.18%
  MAE = 0.240104 | Δ = +0.009388 | Δ% = +3.76%

──────── Summary (Var-Corr) ───────
Vanilla  mean MSE : 0.105316
Ensemble mean MSE : 0.101966
Average Δ MSE     : +0.003349
Average Δ MSE (%) : +3.18%

Vanilla  mean MAE : 0.249491
Ensemble mean MAE : 0.240104
Average

## 1E->1E


In [49]:
results1_ER_test['ES']['mae_test_EES']

0.24949122965335846

In [ ]:
import torch, os, numpy as np
from torch.utils.data import DataLoader, TensorDataset

TOP_K    = 65
CH_CNT   = 1
batch_sz = 8

def infer_one(model, loader, ch):
    buf = []
    model.eval()
    with torch.no_grad():
        for xb, yb, pv in loader:
            xb, pv = xb.to(device), pv.to(device)
            buf.append(model(xb, pv)[:, :, ch].cpu())   # [B,P]
    return torch.cat(buf, 0)    # [N,P]


van_mse_list, ens_mse_list, delta_mse_list, delta_mse_pct_list = [], [], [], []
van_mae_list, ens_mae_list, delta_mae_list, delta_mae_pct_list = [], [], [], []


for ch in range(CH_CNT):

    print(f"\n=== Channel {ch} ===")

    # 1) Vanilla Scores
    v_mse = results1_ER_test['ES']['mse_test_EES']
    v_mae = results1_ER_test['ES']['mae_test_EES']

    print(f"[Vanilla]   MSE : {v_mse:.6f} | MAE : {v_mae:.6f}")

    van_mse_list.append(v_mse)
    van_mae_list.append(v_mae)

    preds_test_arr = []
    ok_segs = []

    for seg in top_seg_idx_E1_val_1E[ch][:TOP_K]:

        ckpt = f"./model2_ER_E1_1E/seg{seg}/ch{ch}/checkpoint.pth"
        if not os.path.exists(ckpt):
            print(f"  [skip] missing checkpoint : seg{seg}")
            continue

        mdl = Model(base_cfg).to(device)
        mdl.load_state_dict(torch.load(ckpt, map_location=device))

        pv_seg = test_pred_segments_E1[:, seg, :, :]
        dl = DataLoader(
            TensorDataset(test_sequences, test_labels, pv_seg),
            batch_size=batch_sz, shuffle=False
        )

        preds_test_arr.append(infer_one(mdl, dl, ch))
        ok_segs.append(seg)

    if len(preds_test_arr) < 5:
        print("  Not enough valid models.")
        continue

    step = 5
    Ks = list(range(step, len(preds_test_arr)+1, step))

    var_list = []
    rho_list = []

    for K in Ks:
        F = torch.stack(preds_test_arr[:K], dim=0)   # [K,N,P]

        V = torch.var(F, dim=0).mean().item()
        var_list.append(V)

        flat = F.reshape(K, -1).numpy()
        corr = np.corrcoef(flat)
        R    = np.mean(np.abs(corr[np.triu_indices(K, 1)]))
        rho_list.append(R)


    V_min, V_max = min(var_list), max(var_list)
    R_min, R_max = min(rho_list), max(rho_list)

    V_norm = [(v - V_min) / (V_max - V_min + 1e-12) for v in var_list]
    R_norm = [(r - R_min) / (R_max - R_min + 1e-12) for r in rho_list]

    alpha, beta = 1, 1.1   # weight

    scores = [alpha*v + beta*r for v, r in zip(V_norm, R_norm)]
    best_K = Ks[int(np.argmin(scores))]

    print("  — Var / |Corr| / Score —")
    for k, v, r, s in zip(Ks, var_list, rho_list, scores):
        tag = "<- pick" if k == best_K else ""
        print(f"   K={k:2d} | Var={v:.3e} | |ρ|={r:.4f} | S={s:.4f} {tag}")

    ens_pred = torch.mean(torch.stack(preds_test_arr[:best_K]), dim=0)  # [N,P]
    y_true   = test_labels[:, -base_cfg.pred_len:, ch].cpu()

    mse_ens = torch.mean((ens_pred - y_true)**2).item()
    mae_ens = torch.mean(torch.abs(ens_pred - y_true)).item()

    ens_mse_list.append(mse_ens)
    ens_mae_list.append(mae_ens)

    delta_mse = v_mse - mse_ens
    delta_mae = v_mae - mae_ens

    delta_mse_pct = (delta_mse / v_mse) * 100
    delta_mae_pct = (delta_mae / v_mae) * 100

    delta_mse_list.append(delta_mse)
    delta_mae_list.append(delta_mae)
    delta_mse_pct_list.append(delta_mse_pct)
    delta_mae_pct_list.append(delta_mae_pct)

    sign = "+" if delta_mse > 0 else "-"
    print(f"[Var-Corr]  K={best_K}")
    print(f"   MSE = {mse_ens:.6f} | Δ = {delta_mse:+.6f} | Δ% = {delta_mse_pct:+.2f}%")
    print(f"   MAE = {mae_ens:.6f} | Δ = {delta_mae:+.6f} | Δ% = {delta_mae_pct:+.2f}%")

if ens_mse_list:
    print("\n──────── Summary (Var-Corr) ───────")
    print(f"Vanilla  mean MSE  : {np.mean(van_mse_list):.6f}")
    print(f"Ensemble mean MSE  : {np.mean(ens_mse_list):.6f}")
    print(f"Average Δ MSE      : {np.mean(delta_mse_list):+.6f}")
    print(f"Average Δ MSE (%)  : {np.mean(delta_mse_pct_list):+.2f}%\n")

    print(f"Vanilla  mean MAE  : {np.mean(van_mae_list):.6f}")
    print(f"Ensemble mean MAE  : {np.mean(ens_mae_list):.6f}")
    print(f"Average Δ MAE      : {np.mean(delta_mae_list):+.6f}")
    print(f"Average Δ MAE (%)  : {np.mean(delta_mae_pct_list):+.2f}%")



=== Channel 0 ===
[Vanilla]   MSE : 0.105316 | MAE : 0.249491


  — Var / |Corr| / Score —
   K= 5 | Var=1.130e-02 | |ρ|=0.9837 | S=1.1000 
   K=10 | Var=1.777e-02 | |ρ|=0.9808 | S=1.1107 
   K=15 | Var=1.966e-02 | |ρ|=0.9775 | S=0.9144 
   K=20 | Var=2.030e-02 | |ρ|=0.9776 | S=0.9504 
   K=25 | Var=2.034e-02 | |ρ|=0.9775 | S=0.9461 
   K=30 | Var=2.064e-02 | |ρ|=0.9778 | S=0.9846 
   K=35 | Var=2.099e-02 | |ρ|=0.9779 | S=1.0031 
   K=40 | Var=2.149e-02 | |ρ|=0.9772 | S=0.9659 
   K=45 | Var=2.324e-02 | |ρ|=0.9754 | S=0.8891 <- pick
   K=50 | Var=2.809e-02 | |ρ|=0.9743 | S=0.9866 
   K=55 | Var=3.044e-02 | |ρ|=0.9739 | S=1.0438 
   K=60 | Var=3.309e-02 | |ρ|=0.9726 | S=1.0450 
   K=65 | Var=3.660e-02 | |ρ|=0.9704 | S=1.0000 
[Var-Corr]  K=45
   MSE = 0.099104 | Δ = +0.006211 | Δ% = +5.90%
   MAE = 0.241473 | Δ = +0.008018 | Δ% = +3.21%

──────── Summary (Var-Corr) ───────
Vanilla  mean MSE  : 0.105316
Ensemble mean MSE  : 0.099104
Average Δ MSE      : +0.006211
Average Δ MSE (%)  : +5.90%

Vanilla  mean MAE  : 0.249491
Ensemble mean MAE  : 0.241473